In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

## Download training and test data

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

In [3]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

## DataLoader
 - `Dataset` is passed as argument to `DataLoader`.
 - This wraps an iterable over dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading.
 - For a batch size of 64. Each element in the dataloader iterable will return a batch of 64 features and lables.

In [ ]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

## Creating Models

In [ ]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda"

print(f"Using {device} device")

In [ ]:
# Define Model
class NeuralNetwork(nn.Module):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.flatten = nn.Flatten()
        self.linear_relu_statck = nn.Sequential(
         nn.Linear(28*28, 512),
         nn.ReLU(),
         nn.Linear(512, 512),
         nn.ReLU(),
         nn.Linear(512, 10)
        )

    
    def forward(self, x):
        print("Input feature, X ",x)
        x = self.flatten(x)
        print("Flatten output :",x)
        logits = self.linear_relu_statck(x)
        print("Sequential relu stack operation :", logits)
        return logits
    

model = NeuralNetwork().to(device)
print("Model :", model)

## Optimizing the Model Parameter
 - To train a model, we need a `loss function` and `optimizer`.

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# In every training loop,
 - the model makes predictions on the training dataset (fed to it in batches)
 - backpropagates the error to adjust the model's parameters.

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        print(f"Batch: {batch + 1}")
        print("X initiall tensor :", X)
        print("y initiall tensor :", y)

        X, y = X.to(device), y.to(device)

        print("X To cpu tensor :", X)
        print("y To cpu tensor :", y)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        print("Prediction ", pred)
        print("Loss ", loss)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

# Models Performance 
 - we also require to measure the model's performance against the test dataset to ensure its learning.

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.eval()

    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            
            pred = model(X)

            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}% , Avg Loss: {test_loss:>8f} \n")

# Training process 
- is conducted over several iterations (*epochs*).
- In each epoch, the model learns parameters to make better predictions.
- We print the model's accuracy and loss at each epoch
- We like to see the accuracy increase and the loss decrease with each epoch.


In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n----------------------------------------------")
    train(test_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("{'#'*10} Done! {'#'*10}")